In [ ]:
print(f'Hello ch04!')

The note book is [here](https://github.com/ageron/handson-ml/blob/master/04_training_linear_models.ipynb)

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 4.1.1 Normal equation

In [ ]:
X = 2 * np.random.rand(100, 1)
print(X.shape, X[:5])
y = 4 + 3 * X + np.random.rand(100, 1)

In [ ]:
fig = plt.figure()
plt.scatter(X, y, alpha=0.5)

In [ ]:
X.shape

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]

In [ ]:
X_b.shape

In [ ]:
?np.dot

In [ ]:
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
print(theta_best)

In [ ]:
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
X_new_b

In [ ]:
y_predict = X_new_b.dot(theta_best)
y_predict

In [ ]:
fig = plt.figure()
plt.scatter(X, y, alpha=0.5)
plt.plot(X_new, y_predict, 'r', label='Prediction')
plt.legend()

#### Use `sklearn` to perform linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
print(lin_reg.intercept_, lin_reg.coef_)

In [ ]:
theta_best_svd, residules, rank, s = np.linalg.lstsq(X_b, y, rcond=None)
print(theta_best_svd)

In [ ]:
np.linalg.pinv(X_b).dot(y)

### 4.2.1 Batch Gradient

In [ ]:
eta = 0.1
n_iterations = 1000
m = 100

In [ ]:
init_theta = np.random.randn(2, 1)
print(f'Initial gradient: {init_theta}')

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True, figsize=(9.6, 3.2))
plt.ylim([0, 14])

for ix, eta in enumerate([0.01, 0.1, 0.5]):
    ax[ix].scatter(X, y, alpha=0.5)
    ax[ix].plot(X_new, X_new_b.dot(init_theta), 'r--')

    theta = init_theta

    for iter in range(n_iterations):
        gradient = 2. / m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradient
        if iter < 10:
            ax[ix].plot(X_new, X_new_b.dot(theta), 'b')
    ax[ix].set_title(f'eta = {eta}')

    print(f'final gradient: {theta}')

### 4.2.2 Stochastic Gradient Descent

In [ ]:
n_epochs = 50
t0, t1 = 5, 50

def learning_schedule(t):
    return t0 / (t + t1)

In [ ]:
init_theta = np.random.randn(2, 1)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
theta = init_theta

fig, ax = plt.subplots(1, 1, figsize=(9.6, 6.4))
ax.scatter(X, y, alpha=0.5)
ax.plot(X_new, X_new_b.dot(init_theta), 'r--')

cost = []

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(100)
        xi = X_b[random_index]
        yi = y[random_index]
        gradients = 2 * (xi.dot(theta) - yi) * xi
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients.reshape(2,1)
        
        if epoch == 0 and i < 20:
            ax.plot(X_new, X_new_b.dot(theta), 'b')
        
        y_predict = X_b.dot(theta)
        cost.append(mean_squared_error(y, y_predict))

print(f'final gradient: {theta}')

#### Plot the cost funtion, should be bumpy

In [ ]:
fig = plt.figure()
plt.plot(cost)
plt.xlim([0,200])
plt.ylim([0,1.0])

#### Use sklearn

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)

In [ ]:
sgd_reg.fit(X, y.ravel())

In [ ]:
sgd_reg.coef_, sgd_reg.intercept_

## 4.3 Polynomial Regression

In [ ]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X ** 2 + X + 2 + np.random.rand(m, 1)

In [ ]:
fig = plt.figure()
plt.rc('lines', markersize=3)
plt.scatter(X, y)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

In [ ]:
print(X[0], X_poly[0])

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

In [ ]:
lin_reg.intercept_, lin_reg.coef_

In [ ]:
X_new = np.arange(-3., 3., 0.1).reshape(-1,1)

In [ ]:
y_pred = lin_reg.predict(poly_features.fit_transform(X_new))

In [ ]:
fig = plt.figure()
plt.rc('lines', markersize=3)
plt.scatter(X, y)
plt.plot(X_new, y_pred, 'r', label='preditions')
plt.xlabel('X1')
plt.legend()